<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/volatilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas_datareader import DataReader as pdr

ffd = pdr('F-F_Research_Data_Factors_daily','famafrench', start=1900)[0]/100

# get daily market return data
mktd = ffd["Mkt-RF"] + ffd.RF
mktd = pd.DataFrame(mktd).reset_index()
mktd.columns = ['date', 'ret']
mktd['month'] = mktd.date.dt.to_period('M').astype(str)

mktd.head(10)

,date,ret,month
0,1926-07-01,0.00109,1926-07
1,1926-07-02,0.00459,1926-07
2,1926-07-06,0.00179,1926-07
3,1926-07-07,0.00099,1926-07
4,1926-07-08,0.00219,1926-07
5,1926-07-09,-0.00701,1926-07
6,1926-07-10,0.00629,1926-07
7,1926-07-12,0.00049,1926-07
8,1926-07-13,0.00489,1926-07
9,1926-07-14,0.00049,1926-07


In [ ]:
# Use daily return data to compute monthly return and volatility of each month
mktm = pd.DataFrame(mktd.groupby('month').ret.std())
mktm.columns = ['vol']
mktm['ret'] = mktd.groupby('month').ret.apply(lambda x: (1+x).prod()-1)
mktm['vol_lag'] = mktm.vol.shift()
mktm = mktm.reset_index()

mktm.head(10)

,month,vol,ret,vol_lag
0,1926-07,0.004528,0.031161,NaN
1,1926-08,0.005926,0.029367,0.004528
2,1926-09,0.004956,0.005894,0.005926
3,1926-10,0.008389,-0.029859,0.004956
4,1926-11,0.003820,0.029049,0.008389
5,1926-12,0.004053,0.029258,0.003820
6,1927-01,0.004151,0.002396,0.004053
7,1927-02,0.002564,0.044594,0.004151
8,1927-03,0.005722,0.003851,0.002564
9,1927-04,0.004896,0.007639,0.005722


In [ ]:
# We can also get compute the monthly volatility and return for individual stock

ticker = 'CVX'
close = pdr(ticker, "yahoo", start=1970)["Adj Close"]
daily = close.pct_change().dropna()
daily = pd.DataFrame(daily).reset_index()
daily.columns = ['date', 'ret']
daily['month'] = daily.date.dt.to_period('M').astype(str)
monthly = pd.DataFrame(daily.groupby('month').ret.std())
monthly.columns = ['vol']
monthly['ret'] = daily.groupby('month').ret.apply(lambda x: (1+x).prod() - 1)
monthly['vol_lag'] = monthly.vol.shift()
monthly = monthly.reset_index()

monthly.head(10)

,month,vol,ret,vol_lag
0,1970-01,0.011653,-0.117224,NaN
1,1970-02,0.019395,0.045856,0.011653
2,1970-03,0.011564,-0.063158,0.019395
3,1970-04,0.009673,-0.061798,0.011564
4,1970-05,0.014991,0.060014,0.009673
5,1970-06,0.013499,-0.074713,0.014991
6,1970-07,0.014868,0.093167,0.013499
7,1970-08,0.015482,0.097150,0.014868
8,1970-09,0.011982,-0.039473,0.015482
9,1970-10,0.011721,0.054795,0.011982
